<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/seventh_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seventh_Test

We begin with the training test, it's important that for the moment we are not going train the model itself. That requires a GPU with almost 50 GB of VRAM.
But we can call the forward pass without problems. I wonder if we can init the model with dummy paremeters, that would be faster?
We need a way to prove that our train is efficent.

In [9]:
from huggingface_hub import login
login()

In [1]:
# git clone ....


fatal: destination path 'XHUMAN' already exists and is not an empty directory.


In [2]:
%cd XHUMAN

/content/XHUMAN


In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 21.32s
Prepared 46 packages in 24.94s
Uninstalled 15 packages in 1.04s
Installed 46 packages in 236ms
 + async-lru==2.0.5
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.0
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.1
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@1d86c9b7f2fc0bbf96db004299cb46161275327f)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.1
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

In [28]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [18]:
!git restore .

In [3]:
DS_ID = "NONHUMAN-RESEARCH/TEST_RECORD_ANNOTATIONS"
PRETRAINED_PATH = "lerobot/pi05_base"

In [4]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

dataset = LeRobotDataset(DS_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
from xhuman.policies.pi05.configuration_pi05 import  PI05Config
device = "cuda"
policy_config = PI05Config(
        # pretrained_path=PRETRAINED_PATH,  # No Set pretrained path so factory loads weights
        device=device,
    )

In [6]:
from xhuman.policies.factory import make_xhuman_policy
policy = make_xhuman_policy(
        cfg=policy_config,
        ds_meta=dataset.meta,
    )

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [11]:
import sys
import importlib

# Polyfill 'imp' for Python 3.12 compatibility
if "imp" not in sys.modules:
    import types
    imp = types.ModuleType("imp")
    imp.reload = importlib.reload
    sys.modules["imp"] = imp

%load_ext autoreload
%autoreload 2


Fix a dependencies problem

In [15]:
from transformers.models.gemma import modeling_gemma

def fixed_extra_repr(self):
    if hasattr(self, "weight"):
        repr_str = f"{tuple(self.weight.shape)}, eps={self.eps}"
    else:
        repr_str = f"eps={self.eps}"
    if getattr(self, "dense", None) is not None:
        repr_str += f", adaptive=True, cond_dim={self.cond_dim}"
    return repr_str

modeling_gemma.GemmaRMSNorm.extra_repr = fixed_extra_repr

In [7]:
from xhuman.policies.pi05.processor_pi05 import (
    make_pi05_pre_post_processors_ki,
)

preprocessor, postprocessor = make_pi05_pre_post_processors_ki(
    config=policy_config,
    dataset_stats=dataset.meta.stats,
)

In [8]:
batch = preprocessor(dataset[0])

In [19]:

fow = policy.forward(batch)

never


RuntimeError: The size of tensor a (1018) must match the size of tensor b (969) at non-singleton dimension 2

In [15]:
import os

# Check current directory
print(f"Current working directory: {os.getcwd()}")

# Define absolute path to the file
file_path = "/content/XHUMAN/xhuman/policies/pi05/modeling_pi_05.py"

if os.path.exists(file_path):
    print(f"\nReading code from: {file_path}")
    # Read lines 130-150 (make_att_2d_masks) and 840-870 (forward)
    with open(file_path, 'r') as f:
        lines = f.readlines()
        print("--- make_att_2d_masks (around line 138) ---")
        for i, line in enumerate(lines[130:150], 131):
            print(f"{i}: {line.rstrip()}")

        print("\n--- forward (around line 858) ---")
        for i, line in enumerate(lines[840:870], 841):
            print(f"{i}: {line.rstrip()}")
else:
    print(f"File not found at: {file_path}")
    # Search for it
    print("Searching for modeling_pi_05.py...")
    for root, dirs, files in os.walk("/content"):
        if "modeling_pi_05.py" in files:
            print(f"Found at: {os.path.join(root, 'modeling_pi_05.py')}")

# Inspect policy config to check expected image size vs actual
print("\nPolicy Config:")
try:
    # Check if vision_config exists and has image_size
    if hasattr(policy.model.config, "vision_config"):
        print(f"Vision Config Image Size: {policy.model.config.vision_config.image_size}")
        print(f"Vision Config Patch Size: {policy.model.config.vision_config.patch_size}")

    # Check number of image tokens usually expected
    # text_tokens = 200
    # 3 images * (image_size/patch_size)^2 ?
except Exception as e:
    print(f"Could not print config details: {e}")

# Inspect preprocessor transforms to see if resizing is happening
print("\nPreprocessor Transforms:")
try:
    print(preprocessor.transforms)
except Exception as e:
    print(e)

Current working directory: /content

Reading code from: /content/XHUMAN/xhuman/policies/pi05/modeling_pi_05.py
--- make_att_2d_masks (around line 138) ---
131:     cumsum = torch.cumsum(att_masks, dim=1)
132:     # att_2d_masks shape: (B, N, N)
133:     att_2d_masks = cumsum[:, None, :] <= cumsum[:, :, None]
134:     # pad_2d_masks shape: (B, N, N)
135:     pad_2d_masks = pad_masks[:, None, :] * pad_masks[:, :, None]
136:     # att_2d_masks & pad_2d_masks shape: (B, N, N)
137:     return att_2d_masks & pad_2d_masks
138: 
139: 
140: def pad_vector(vector, new_dim):
141:     """Pad the last dimension of a vector to new_dim with zeros.
142: 
143:     Can be (batch_size x sequence_length x features_dimension)
144:     or (batch_size x features_dimension)
145:     """
146:     if vector.shape[-1] >= new_dim:
147:         return vector
148:     return F.pad(vector, (0, new_dim - vector.shape[-1]))
149: 
150: 

--- forward (around line 858) ---
841:         )
842:         suffix_embs, suffix_

In [19]:
!git diff

In [17]:
!git restore .

In [18]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
